In [1]:
%matplotlib inline

In [2]:
import os
import glob

import numpy as np
import pandas as pd

import tensorflow as tf

from typing import *
from collections import Counter

import sklearn
from sklearn import model_selection


In [3]:

def get_text_data(input_dir):
    """
    Purpose: preprocess liwc and nrc
    Input
        input_dir {string} : path to input_directory (ex, "~/Train")
    Output:
        id_list {numpy array of strings}: array of user ids sorted alphabetically,
                                        to determine order of features and labels DataFrames
        text_data {pandas DataFrame of float}: unscaled text data (liwc and nrc combined)
    """
    # Load and sort text data
    liwc = pd.read_csv(os.path.join(input_dir, "Text", "liwc.csv"), sep = ',')
    liwc = liwc.sort_values(by=['userId'])

    nrc = pd.read_csv(os.path.join(input_dir, "Text", "nrc.csv"), sep = ',')
    nrc = nrc.sort_values(by=['userId'])

    # Build list of subject ids ordered alphabetically
    # Check if same subject lists in both sorted DataFrames (liwc and nrc)
    if np.array_equal(liwc['userId'], nrc['userId']):
        id_list = liwc['userId'].to_numpy()
    else:
        raise Exception('userIds do not match between liwc and nrc data')

    # merge liwc and nrc DataFrames using userId as index
    liwc.set_index('userId', inplace=True)
    nrc.set_index('userId', inplace=True)

    text_data = pd.concat([liwc, nrc], axis=1, sort=False)

    return id_list, text_data


def get_image_clean(sub_ids, oxford, means):
    '''
    Purpose: preprocess oxford metrics derived from profile pictures (part 2)
    Input:
        sub_ids {numpy array of strings}: ordered list of userIDs
        oxford {pandas DataFrame of floats}: unscaled oxford features of users with 1+ face
        means {list of float}: mean values for each feature averaged from train set,
                    to replace missing values for userids with no face (train and test set)
    Output:
        image_data {pandas DataFrame of float}: unscaled oxford image data
                with mean values replacing missing entries
    '''
    # list of ids with at least one face on image: 7174 out of 9500 in train set
    ox_list = np.sort(oxford['userId'].unique(), axis=None)
    # list of ids in text_list who have no face metrics in oxford.csv (2326 in train set)
    ox_noface = np.setdiff1d(sub_ids, ox_list)

    # Create DataFrame for userids with no face (1 row per userid)
    # values are mean metrics averaged from users with entries (training set)
    ox_nf = pd.DataFrame(ox_noface, columns = ['userId'])
    columns = oxford.columns[2:].tolist()
    for column, mean in zip(columns, means):
        ox_nf.insert(loc=ox_nf.shape[1], column=column, value=mean, allow_duplicates=True)
    # insert column 'noface' = 1 if no face in image, else 0
    ox_nf.insert(loc=ox_nf.shape[1], column='noface', value=1, allow_duplicates=True)
    # insert column 'multiface' = 1 if many faces in image, else 0
    ox_nf.insert(loc=ox_nf.shape[1], column='multiface', value=0, allow_duplicates=True)
    ox_nf.set_index('userId', inplace=True)

    # Format DataFrame from userids with 1+ face
    # insert column 'noface' = 1 if no face in image, else 0
    oxford.insert(loc=oxford.shape[1], column='noface', value=0, allow_duplicates=True)
    # list userIds with multiple faces (714 in train set)
    ox_multiples = oxford['userId'][oxford['userId'].duplicated()].tolist()
    # insert column 'multiface' = 1 if many faces in image, else 0
    oxford.insert(loc=oxford.shape[1], column='multiface', value=0, allow_duplicates=True)
    multi_mask = pd.Series([uid in ox_multiples for uid in oxford['userId']])
    i = oxford[multi_mask].index
    oxford.loc[i, 'multiface'] = 1
    # drop duplicate entries with same userId (keep first entry per userId)
    oxford.drop_duplicates(subset ='userId', keep='first', inplace=True)

    # merge the two DataFrames
    oxford.drop(['faceID'], axis=1, inplace=True)
    oxford.set_index('userId', inplace=True)
    image_data = pd.concat([ox_nf, oxford], axis=0, sort=False).sort_values(by=['userId'])

    if not np.array_equal(image_data.index, sub_ids):
        raise Exception('userIds do not match between oxford file and id list')

    return image_data


def get_image_raw(data_dir):
    '''
    Purpose: preprocess oxford metrics derived from profile pictures (part 1)
    Input
        input_dir {string} : path to input_directory (ex, "~/Train")
    Output:
        image_data {pandas DataFrame of float}: unscaled oxford image data
    '''
    # Load data of oxford features extracted from profile picture (face metrics)
    # 7915 entries; some users have no face, some have multiple faces on image.
    # userids with 1+ face on image: 7174 out of 9500 (train set)
    # duplicated entries (userids with > 1 face on same image): 741 in train set
    oxford = pd.read_csv(os.path.join(data_dir, "Image", "oxford.csv"), sep = ',')
    #oxford = oxford.sort_values(by=['userId'])
    '''
    NOTE: headPose_pitch has NO RANGE, drop that feature
    '''
    oxford.drop(['headPose_pitch'], axis=1, inplace=True)

    return oxford


def get_likes_kept(data_dir, num_features) -> List[str]:
    '''
    Purpose: get list of likes to keep as features
    Input:
        data_dir {str} : the parent input directory
        num_features {int} : the number of likes to keep as features,
                        starting from those with highest frequencies
    Output:
        freq_like_id {List of strings}: frequency of most frequent likes,
                    (number = num_features), in descending ordered, indexed by like_id
    '''
    #Why return frequency?
    relation = pd.read_csv(os.path.join(data_dir, "Relation", "Relation.csv")) #, index_col=1)
    relation = relation.drop(['Unnamed: 0'], axis=1)
    like_ids_to_keep = relation['like_id'].value_counts(sort=True, ascending=False)[:num_features] #This sorts features by frequency

    #sort like indices (which are the keys associated with the values kepts)
    likes_int64_list = sorted(like_ids_to_keep.keys()) # This sorts indices by like_id
    likes_str_list = [str(l) for l in likes_int64_list]
    return likes_str_list


def get_relations(data_dir: str, sub_ids: List[str], like_ids_to_keep: List[str]):
    '''
    Purpose: preprocess relations dataset ('likes')

    Input:
        data_dir {str} -- the parent input directory
        sub_ids {numpy array of strings} -- the ordered list of userids
        like_ids_to_keep {List[str]} -- The list of page IDs to keep.

    Returns:
        relations_data -- multihot matrix of the like_id. Rows are indexed with userid, entries are boolean.
    '''
    relation = pd.read_csv(os.path.join(data_dir, "Relation", "Relation.csv")) #, index_col=1)
    relation = relation.drop(['Unnamed: 0'], axis=1)

    ## One HUGE step:
    # likes_to_keep = like_ids_to_keep.keys()
    # kept_relations = relation[relation.like_id.isin(likes_to_keep)]
    # multi_hot_relations = pd.get_dummies(kept_relations, columns=["like_id"], prefix="")
    # multi_hot = multi_hot_relations.groupby(("userid")).sum()
    # return multi_hot_relations
    ###
    total_num_pages = len(like_ids_to_keep)
    # Create a multihot likes matrix of booleans (rows = userids, cols = likes), by batch
    batch_size = 1000

    # Create empty DataFrame with sub_ids as index list
    relation_data = pd.DataFrame(sub_ids, columns = ['userid'])
    relation_data.set_index('userid', inplace=True)

    for start_index in range(0, total_num_pages, batch_size):
        end_index = min(start_index + batch_size, total_num_pages)

        # sets are better for membership testing than lists.
        like_ids_for_this_batch = set(like_ids_to_keep[start_index:end_index])

        filtered_table = relation[relation['like_id'].isin(like_ids_for_this_batch)]
        ## THIS is the slow part:
        relHot = pd.get_dummies(filtered_table, columns=['like_id'], prefix="", prefix_sep="")
        ##
        relHot = relHot.groupby(['userid']).sum().astype(float) # this makes userid the index

        relation_data = pd.concat([relation_data, relHot], axis=1, sort=True)

    relation_data = relation_data.reindex(like_ids_to_keep, axis=1)
    relation_data.fillna(0.0, inplace=True)
    relation_data = relation_data.astype("bool")

    # will be different if users in relation.csv are not in sub_ids
    if not np.array_equal(relation_data.index, sub_ids):
        raise Exception(f"""userIds do not match between relation file and id list:
    {relation_data.index}
    {sub_ids}

    """)

    return relation_data

def get_likes_lists(likes_data, max_num_likes):
    '''
    Purpose: make list of lists of indices of liked pages per user
    Input:
        likes_data {pandas DataFrame}: multihot matrix of the like_id. Rows are indexed with userid, entries are boolean
    Output:
        lists_of_likes {list of lists of int}: indices of pages liked by each user,
                padded with zeros to lenght = max_num_likes

    '''
    # create list of lists of indices (one per user) corresponding to liked pages in one-hot matrix
    index_lists = []
    for index in likes_data.index:
        likes_indices = np.nonzero(likes_data.loc[index].to_numpy())[0].tolist()
        index_lists.append(likes_indices)

    # pad each list of indices with 0s to set lenght = max_num_likes
    lists_padded = tf.keras.preprocessing.sequence.pad_sequences(index_lists,
    padding='post', maxlen=max_num_likes)

    lists_of_likes = pd.DataFrame(lists_padded)

    lists_of_likes.insert(loc=lists_of_likes.shape[1], column='userid', value=likes_data.index, allow_duplicates=True)
    lists_of_likes.set_index('userid', inplace=True)

    return lists_of_likes

def make_label_dict(labels):
    '''
    Purpose: make dictionnary of labels from pandas DataFrame
    Input:
        labels {pandas DataFrame}: labels ordered per userids (alphabetical order)
    Output:
        labels_dict {dictionary of pandas DataFrames}: labels (one entry per metric) ordered alphabetically
                by userid for the training set, with userids as index.

    '''
    gender = labels['gender']

    age_grps = labels[['age_xx_24', 'age_25_34', 'age_35_49', 'age_50_xx']]

    '''
    Note: : each DataFrames (value) is indexed by userid in labels_dict
    '''
    labels_dict = {}
    labels_dict['userid'] = labels.index
    labels_dict['gender'] = gender
    labels_dict['age_grps'] = age_grps
    labels_dict['ope'] = labels['ope']
    labels_dict['con'] = labels['con']
    labels_dict['ext'] = labels['ext']
    labels_dict['agr'] = labels['agr']
    labels_dict['neu'] = labels['neu']

    return labels_dict


def preprocess_labels(data_dir, sub_ids):
    '''
    Purpose: preprocess entry labels from training set
    Input:
        datadir {string} : path to training data directory
        sub_ids {numpy array of strings}: list of subject ids ordered alphabetically
    Output:
        labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.

    '''
    labels = pd.read_csv(os.path.join(data_dir, "Profile", "Profile.csv"))

    def age_group_id(age_str: str) -> int:
        """Returns the age group category ID (an integer from 0 to 3) for the given age (string)

        Arguments:
            age_str {str} -- the age

        Returns:
            int -- the ID of the age group: 0 for xx-24, 1 for 25-34, 2 for 35-49 and 3 for 50-xx.
        """
        age = int(age_str)
        if age <= 24:
            return 0
        elif age <= 34:
            return 1
        elif age <= 49:
            return 2
        else:
            return 3

    labels = labels.assign(age_group = lambda dt: pd.Series([age_group_id(age_str) for age_str in dt["age"]]))
    # labels = labels.assign(age_xx_24 = lambda dt: pd.Series([int(age) <= 24 for age in dt["age"]]))
    # labels = labels.assign(age_25_34 = lambda dt: pd.Series([25 <= int(age) <= 34 for age in dt["age"]]))
    # labels = labels.assign(age_35_49 = lambda dt: pd.Series([35 <= int(age) <= 49 for age in dt["age"]]))
    # labels = labels.assign(age_50_xx = lambda dt: pd.Series([50 <= int(age) for age in dt["age"]]))

    labels = labels.sort_values(by=['userid'])
    # check if same subject ids in labels and sub_ids
    if not np.array_equal(labels['userid'].to_numpy(), sub_ids):
        raise Exception('userIds do not match between profiles labels and id list')

    labels = labels.drop(['Unnamed: 0'], axis=1)
    labels.set_index('userid', inplace=True)

    return labels


def preprocess_train(data_dir, num_likes=10_000, max_num_likes=2145):
    '''
    Purpose: preprocesses training dataset (with labels) and returns scaled features,
    labels and parameters to scale the test data set
    Input
        data_dir {string}: path to ~/Train data directory
        num_likes {int}: number of like_ids to keep as features
        max_num_likes {int}: maximum number of pages liked by a single user
    Output:
        train_features {pandas DataFrame}: vectorized features scaled between 0 and 1
                for each user id in the training set, concatenated for all modalities
                (order = text + image + relation), with userid as DataFrame index.
        **(updated:)features_q10_q90 {tupple of 2 pandas Series}: series of 10th and 90th quantile values of
                text + image features from train dataset, to be used to scale test data.
                Note that the multihot relation features do not necessitate scaling.
        image_means {list of float}: means from oxford dataset to replace missing entries in oxford test set
        likes_kept {list of strings}: ordered likes_ids to serve as columns for test set relation features matrix
        train_labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.

    TO CONSIDER: convert outputted pandas to tensorflow tf.data.Dataset...
    https://www.tensorflow.org/guide/data
    '''
    # sub_ids: a numpy array of subject ids ordered alphabetically.
    # text_data: a pandas DataFrame of unscaled text data (liwc and nrc)
    sub_ids, text_data = get_text_data(data_dir)
    # image_data: pandas dataframe of oxford data
    # image_min_max: a tupple of 2 pandas series, the min and max values from oxford training features
    image_data_raw = get_image_raw(data_dir)
    image_means = image_data_raw.iloc[:, 2:].mean().tolist()
    image_data = get_image_clean(sub_ids, image_data_raw, image_means)

    '''
    Note: Scale the text and image data BEFORE concatenating with relations
    Update: scaling w RobustScaler rather than MinMaxScaler algo, due to outliers
    '''
    features_to_scale = pd.concat([text_data, image_data.iloc[:, :-2]], axis=1, sort=False)
    #feat_min = features_to_scale.min()
    #feat_max = features_to_scale.max()
    feat_q10 = features_to_scale.quantile(q = 0.10)
    feat_q90 = features_to_scale.quantile(q = 0.90)

    #feat_scaled = (features_to_scale - feat_min) / (feat_max - feat_min)
    #features_min_max = (feat_min, feat_max)
    feat_scaled = (features_to_scale - feat_q10) / (feat_q90 - feat_q10)
    features_q10_q90 = (feat_q10, feat_q90)

    likes_kept = get_likes_kept(data_dir, num_likes)

    # multi-hot matrix of likes from train data
    likes_data = get_relations(data_dir, sub_ids, likes_kept)

    train_likes_lists = get_likes_lists(likes_data, max_num_likes)

    # concatenate all scaled features into a single DataFrame
    additional_weird_features = image_data.iloc[:, -2:]
    train_features = pd.concat([feat_scaled, additional_weird_features, train_likes_lists], axis=1, sort=False)

    # DataFrame of training set labels
    train_labels = preprocess_labels(data_dir, sub_ids)


    #return train_features, features_min_max, image_means, likes_kept, train_labels
    return train_features, features_q10_q90, image_means, likes_kept, train_labels


#def preprocess_test(data_dir, min_max_train, image_means_train, likes_kept_train):
def preprocess_test(data_dir, q10_q90_train, image_means_train, likes_kept_train, max_num_likes=2145):
    '''
    Purpose: preprocesses test dataset (no labels)
    Input:
        datadir {string}: path to Test data directory
        (**updated)q10_q90_train {tupple of two numpy arrays}: 10th and 90th quantile values for
                concatenated text and image features (from train set)
        image_means_train {list of float}: means from oxford training dataset to replace
                missing entries in oxford test set
        likes_kept_train {list of strings}: most frequent likes_ids from train set
                (ordered by frequency) to serve as columns in relation features matrix
        max_num_likes {int}: maximum number of pages liked by a single user (from train set)
    Output:
        test_features {pandas DataFrame}: vectorized features of test set

    '''
    # sub_ids: a numpy array of subject ids ordered alphabetically.
    # text_data: a pandas DataFrame of unscaled text data (liwc and nrc)
    sub_ids, text_data = get_text_data(data_dir)

    # image_data: pandas dataframe of oxford data
    # image_min_max: a tupple of 2 pandas series, the min and max values from oxford training features
    image_data_raw = get_image_raw(data_dir)
    image_data = get_image_clean(sub_ids, image_data_raw, image_means_train)

    '''
    Note: Scale the text and image data BEFORE concatenating with relations
    '''
    features_to_scale = pd.concat([text_data, image_data.iloc[:, :-2]], axis=1, sort=False)
    #feat_min = min_max_train[0]
    #feat_max = min_max_train[1]
    feat_q10 = q10_q90_train[0]
    feat_q90 = q10_q90_train[1]

    #feat_scaled = (features_to_scale - feat_min) / (feat_max - feat_min)
    feat_scaled = (features_to_scale - feat_q10) / (feat_q90 - feat_q10)

    # multi-hot matrix of likes from train data
    likes_data = get_relations(data_dir, sub_ids, likes_kept_train)

    # list of lists of indices corresponding to pages liked
    # each padded with 0s (list's max length = max_num_likes)
    test_likes_lists = get_likes_lists(likes_data, max_num_likes)

    # concatenate all scaled features into a single DataFrame
    test_features = pd.concat([feat_scaled, image_data.iloc[:, -2:], test_likes_lists], axis=1, sort=False)

    return test_features


def get_train_val_sets(features, labels, val_prop):
    '''
    Purpose: Splits training dataset into a train and a validation set of
    ratio determined by val_prop (x = features, y = labels)
    Input
        features {pandas DataFrame}: vectorized features scaled between 0 and 1
                for each user id in the training set, concatenated for all modalities
                (order = text + image + relation), with userid as DataFrame index.
        labels {pandas DataFrame}: labels ordered by userid (alphabetically)
                for the training set, with userids as index.
        val_prop {float between 0 and 1}: proportion of sample in validation set
                    (e.g. 0.2 = 20% validation, 80% training)
    Output:
        x_train, x_val {pandas DataFrames}: vectorized features for train and validation sets
        y_train, y_val {pandas DataFrames}: train and validation set labels

    TO DO: convert outputted pandas to tensorflow tf.data.Dataset?...
    https://www.tensorflow.org/guide/data
    '''
    # NOTE: UNUSED
    from sklearn import model_selection
    x_train, x_val, y_train, y_val = model_selection.train_test_split(
        features, # training features to split
        labels, # training labels to split
        test_size = val_prop, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
        shuffle= True,
        #stratify = y_data[:1],
        # random_state = 42  # can use to always obtain the same train/validation split
        )

    return x_train, x_val, y_train, y_val


In [4]:
# to preprocess the training dataset:
# 1. set path to Train directory
# 2. call preprocess_train

train_path = '../Train' #modify if working from other directory

train_features, features_q10_q90, image_means, likes_kept, train_labels = preprocess_train(train_path, num_likes=10_000, max_num_likes=2000)


In [5]:
train_features_200, features_q10_q90_200, image_means_200, likes_kept_200, train_labels_200 = preprocess_train(train_path, num_likes=10_000, max_num_likes=200)


In [6]:
# Try different number of max liked pages per user

train_features_100, features_q10_q90_100, image_means_100, likes_kept_100, train_labels_100 = preprocess_train(train_path, num_likes=10_000, max_num_likes=100)
train_features_500, features_q10_q90_500, image_means_500, likes_kept_500, train_labels_500 = preprocess_train(train_path, num_likes=10_000, max_num_likes=500)
train_features_1000, features_q10_q90_1000, image_means_1000, likes_kept_1000, train_labels_1000 = preprocess_train(train_path, num_likes=10_000, max_num_likes=1000)



In [5]:
## INSERT: save (as .csv) the features needed as arguments for preprocess_test in submission; 
# load in test script to feed model
# save tensorflow models in submissions
# https://www.tensorflow.org/guide/saved_model

In [8]:
test_features = preprocess_test(train_path, features_q10_q90, image_means, likes_kept, max_num_likes=2000)


In [9]:
np.sum(train_features.columns == test_features.columns)/test_features.shape[1]

1.0

In [121]:
# Split train and test set for Age Group Classifier

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    train_features, # training features to split
    train_labels, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels['age_group']
    # random_state = 42  # can use to always obtain the same train/validation split
    )

x_train_100, x_val_100, y_train_100, y_val_100 = model_selection.train_test_split(
    train_features_100, # training features to split
    train_labels_100, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels_100['age_group']
    # random_state = 42  # can use to always obtain the same train/validation split
    )

x_train_200, x_val_200, y_train_200, y_val_200 = model_selection.train_test_split(
    train_features_200, # training features to split
    train_labels_200, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels_200['age_group']
    # random_state = 42  # can use to always obtain the same train/validation split
    )

x_train_500, x_val_500, y_train_500, y_val_500 = model_selection.train_test_split(
    train_features_500, # training features to split
    train_labels_500, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels_500['age_group']
    # random_state = 42  # can use to always obtain the same train/validation split
    )

x_train_1000, x_val_1000, y_train_1000, y_val_1000 = model_selection.train_test_split(
    train_features_1000, # training features to split
    train_labels_1000, # training labels to split
    test_size = 0.2, # between 0 and 1, proportion of sample in validation set (e.g., 0.2)
    shuffle= True,
    stratify = train_labels_1000['age_group']
    # random_state = 42  # can use to always obtain the same train/validation split
    )


In [122]:
# Hyper-parameters model age with embedded likes

#batch_size=64
num_layers=2
dense_units=62
#activation='tanh'
#optimizer='sgd'
learning_rate=0.0001
l1_reg=0.0025
l2_reg=0.005
#num_like_pages=5000
#use_dropout=True
dropout_rate=0.1
#use_batchnorm=False

#age_weights = [0.42100598, 0.98445596, 2.27817746, 5.88235294]
    
num_text_features = 91
num_image_features = 65 # added back noface and multiface    
#num_like_features = 8


In [123]:
# calculating weights for age categories w sklearn
#https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html

a_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes = np.unique(y_train['age_group']), y= y_train['age_group'])

age_weights_dict = {}

for i in range(len(a_weights)):
    age_weights_dict[i] = a_weights[i]
    
print(age_weights_dict)   

age_weights_dict_mild = {}
age_weights_dict_mild[0] = 0.5
age_weights_dict_mild[1] = 1.0
age_weights_dict_mild[2] = 1.5
age_weights_dict_mild[3] = 2.0

print(age_weights_dict_mild)

age_weights_dict_first3 = {}
age_weights_dict_first3[0] = 1.0
age_weights_dict_first3[1] = 2.0
age_weights_dict_first3[2] = 3.0
age_weights_dict_first3[3] = 0.1

print(age_weights_dict_first3)


{0: 0.4189636163175303, 1: 0.9890681936491411, 2: 2.272727272727273, 3: 6.1688311688311686}
{0: 0.5, 1: 1.0, 2: 1.5, 3: 2.0}
{0: 1.0, 1: 2.0, 2: 3.0, 3: 0.1}


In [124]:
# model category age using embedding for likes

max_len = 2000

image_features = tf.keras.Input([num_image_features], dtype=tf.float32, name="image_features")
text_features  = tf.keras.Input([num_text_features], dtype=tf.float32, name="text_features")
likes_features = tf.keras.Input([max_len], dtype=tf.int32, name="likes_features")

likes_embedding_block = tf.keras.Sequential(name="likes_embedding_block")
likes_embedding_block.add(tf.keras.layers.Embedding(10000, 8, input_length=max_len,
                                                    #embeddings_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001),
                                                   mask_zero=True))

### GlobalAveragePooling1D averages the embeddings of all pages liked by a user
#likes_embedding_block.add(tf.keras.layers.GlobalAveragePooling1D())

likes_embedding_block.add(tf.keras.layers.Flatten())

## Experimenting: Not helping at all... just learning the baseline
# Trying to reduce contribution of likes to balance out input modalities
#likes_embedding_block.add(tf.keras.layers.Dense(
#    units = 128, activation= 'tanh', 
#    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
## Experimenting

condensed_likes = likes_embedding_block(likes_features)

dense_layers = tf.keras.Sequential(name="dense_layers")
dense_layers.add(tf.keras.layers.Concatenate())
for i in range(num_layers):
    dense_layers.add(tf.keras.layers.Dense(
        units=dense_units,
        activation= 'tanh', #'tanh',
        kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg),      
        ))
        
    dense_layers.add(tf.keras.layers.Dropout(dropout_rate))
    
features = dense_layers([text_features, image_features, condensed_likes])

age_group = tf.keras.layers.Dense(units=4, activation="softmax", name="age_group")(features)

model_age = tf.keras.Model(
    inputs=[text_features, image_features, likes_features],
    outputs= age_group
)    

model_age.compile(
    optimizer = tf.keras.optimizers.get({"class_name": "ADAM", #'ADAM'
                               "config": {"learning_rate": 0.0001}}),    
    loss = 'categorical_crossentropy',
    metrics = ['acc', 'categorical_accuracy']
)

print(model_age.summary())


Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
likes_features (InputLayer)     [(None, 2000)]       0                                            
__________________________________________________________________________________________________
text_features (InputLayer)      [(None, 91)]         0                                            
__________________________________________________________________________________________________
image_features (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
likes_embedding_block (Sequenti (None, 16000)        80000       likes_features[0][0]             
___________________________________________________________________________________________

In [125]:
x_train_txt = x_train.iloc[:, :91].values
x_train_img = x_train.iloc[:, 91:156].values
x_train_lik = x_train.iloc[:, 156:].values

x_val_txt = x_val.iloc[:, :91].values
x_val_img = x_val.iloc[:, 91:156].values
x_val_lik = x_val.iloc[:, 156:].values

y_train_age = tf.keras.utils.to_categorical(y_train['age_group'].values)

y_val_age = tf.keras.utils.to_categorical(y_val['age_group'].values)


In [126]:
'''
history_age_balanced = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=64, epochs=100, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     #validation_split=0.2, 
                                     class_weight=age_weights_dict
                                    )

'''
history_age_balanced_mild = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=64, epochs=55, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     class_weight=age_weights_dict_first3
                                    )

# validation baseline for age group:
print(y_val['age_group'].value_counts()[0]/y_val.shape[0])

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm


Train on 7600 samples, validate on 1900 samples
Epoch 1/55
7600/7600 [==============================] - 3s 431us/sample - loss: 16.2611 - acc: 0.4787 - categorical_accuracy: 0.4787 - val_loss: 7.3069 - val_acc: 0.5921 - val_categorical_accuracy: 0.5921
Epoch 2/55
7600/7600 [==============================] - 2s 252us/sample - loss: 4.0962 - acc: 0.5612 - categorical_accuracy: 0.5612 - val_loss: 2.9153 - val_acc: 0.5963 - val_categorical_accuracy: 0.5963
Epoch 3/55
7600/7600 [==============================] - 2s 250us/sample - loss: 2.8442 - acc: 0.5611 - categorical_accuracy: 0.5611 - val_loss: 2.7698 - val_acc: 0.5826 - val_categorical_accuracy: 0.5826
Epoch 4/55
7600/7600 [==============================] - 2s 248us/sample - loss: 2.7214 - acc: 0.5704 - categorical_accuracy: 0.5704 - val_loss: 2.6593 - val_acc: 0.5826 - val_categorical_accuracy: 0.5826
Epoch 5/55
7600/7600 [==============================] - 2s 251us/sample - loss: 2.6119 - acc: 0.5836 - categorical_accuracy: 0.5836 - v

Epoch 41/55
7600/7600 [==============================] - 2s 252us/sample - loss: 1.3386 - acc: 0.6734 - categorical_accuracy: 0.6734 - val_loss: 1.3959 - val_acc: 0.6453 - val_categorical_accuracy: 0.6453
Epoch 42/55
7600/7600 [==============================] - 2s 248us/sample - loss: 1.3228 - acc: 0.6813 - categorical_accuracy: 0.6813 - val_loss: 1.3821 - val_acc: 0.6384 - val_categorical_accuracy: 0.6384
Epoch 43/55
7600/7600 [==============================] - 2s 254us/sample - loss: 1.3121 - acc: 0.6809 - categorical_accuracy: 0.6809 - val_loss: 1.3813 - val_acc: 0.6521 - val_categorical_accuracy: 0.6521
Epoch 44/55
7600/7600 [==============================] - 2s 251us/sample - loss: 1.2951 - acc: 0.6899 - categorical_accuracy: 0.6899 - val_loss: 1.3672 - val_acc: 0.6595 - val_categorical_accuracy: 0.6595
Epoch 45/55
7600/7600 [==============================] - 2s 251us/sample - loss: 1.2853 - acc: 0.6908 - categorical_accuracy: 0.6908 - val_loss: 1.3589 - val_acc: 0.6458 - val_cate

array([[926, 202,   6,   0],
       [145, 276,  59,   0],
       [ 14, 111,  84,   0],
       [  7,  32,  38,   0]])

In [28]:
#model_age.save('saved_models/age_model_embedding_2000.h5')


In [ ]:
## Training notes: 
# Without rebalancing of weights, at learning rate 0.0001, starts overfitting (loss increasing) 
# around epoch 70

# More balanced classification: batches 16, 30 epochs, reweight 1, 2, 3, 0.1
# For 64 size batches, loss increases around 55... still good classification around 70

## Better training: 64 batches, 55 epochs, reweight 1, 2, 3, 0.1


In [86]:

### PAS CELUI-LA!!
'''
history_age = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                            shuffle=True, batch_size=64, epochs=70, verbose=1, 
                            validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age)
                           )

# baseline for age group:
print(y_val['age_group'].value_counts()[0]/y_val.shape[0])

#score_age = model_age.evaluate([x_val_txt, x_val_img, x_val_lik], y_val_age, verbose=1)

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm
'''


Train on 7600 samples, validate on 1900 samples
Epoch 1/70
7600/7600 [==============================] - 3s 452us/sample - loss: 31.9606 - acc: 0.5414 - categorical_accuracy: 0.5414 - val_loss: 13.2020 - val_acc: 0.5816 - val_categorical_accuracy: 0.5816
Epoch 2/70
7600/7600 [==============================] - 2s 264us/sample - loss: 7.0353 - acc: 0.5792 - categorical_accuracy: 0.5792 - val_loss: 4.9713 - val_acc: 0.5974 - val_categorical_accuracy: 0.5974
Epoch 3/70
7600/7600 [==============================] - 2s 264us/sample - loss: 4.7719 - acc: 0.5913 - categorical_accuracy: 0.5913 - val_loss: 4.5375 - val_acc: 0.6011 - val_categorical_accuracy: 0.6011
Epoch 4/70
7600/7600 [==============================] - 2s 263us/sample - loss: 4.3552 - acc: 0.5972 - categorical_accuracy: 0.5972 - val_loss: 4.1439 - val_acc: 0.6026 - val_categorical_accuracy: 0.6026
Epoch 5/70
7600/7600 [==============================] - 2s 264us/sample - loss: 3.9794 - acc: 0.5987 - categorical_accuracy: 0.5987 - 

Epoch 41/70
7600/7600 [==============================] - 2s 263us/sample - loss: 1.0652 - acc: 0.6105 - categorical_accuracy: 0.6105 - val_loss: 1.0612 - val_acc: 0.6058 - val_categorical_accuracy: 0.6058
Epoch 42/70
7600/7600 [==============================] - 2s 259us/sample - loss: 1.0603 - acc: 0.6095 - categorical_accuracy: 0.6095 - val_loss: 1.0579 - val_acc: 0.6047 - val_categorical_accuracy: 0.6047
Epoch 43/70
7600/7600 [==============================] - 2s 264us/sample - loss: 1.0567 - acc: 0.6109 - categorical_accuracy: 0.6109 - val_loss: 1.0547 - val_acc: 0.6058 - val_categorical_accuracy: 0.6058
Epoch 44/70
7600/7600 [==============================] - 2s 264us/sample - loss: 1.0544 - acc: 0.6126 - categorical_accuracy: 0.6126 - val_loss: 1.0521 - val_acc: 0.6068 - val_categorical_accuracy: 0.6068
Epoch 45/70
7600/7600 [==============================] - 2s 265us/sample - loss: 1.0529 - acc: 0.6120 - categorical_accuracy: 0.6120 - val_loss: 1.0500 - val_acc: 0.6074 - val_cate

array([[1091,   29,   14,    0],
       [ 420,   44,   16,    0],
       [ 153,   32,   24,    0],
       [  48,   18,   11,    0]])

In [105]:
### Try different max number of liked pages per user!!

# model category age using embedding for likes

max_len = 100

image_features = tf.keras.Input([num_image_features], dtype=tf.float32, name="image_features")
text_features  = tf.keras.Input([num_text_features], dtype=tf.float32, name="text_features")
likes_features = tf.keras.Input([max_len], dtype=tf.int32, name="likes_features")

likes_embedding_block = tf.keras.Sequential(name="likes_embedding_block")
likes_embedding_block.add(tf.keras.layers.Embedding(10000, 8, input_length=max_len,
                                                    #embeddings_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001),
                                                   mask_zero=True))

### GlobalAveragePooling1D averages the embeddings of all pages liked by a user
#likes_embedding_block.add(tf.keras.layers.GlobalAveragePooling1D())

likes_embedding_block.add(tf.keras.layers.Flatten())

## Experimenting: Not helping at all... just learning the baseline
# Trying to reduce contribution of likes to balance out input modalities
#likes_embedding_block.add(tf.keras.layers.Dense(
#    units = 128, activation= 'tanh', 
#    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
## Experimenting

condensed_likes = likes_embedding_block(likes_features)

dense_layers = tf.keras.Sequential(name="dense_layers")
dense_layers.add(tf.keras.layers.Concatenate())
for i in range(num_layers):
    dense_layers.add(tf.keras.layers.Dense(
        units=dense_units,
        activation= 'tanh', #'tanh',
        kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg),      
        ))
        
    dense_layers.add(tf.keras.layers.Dropout(dropout_rate))
    
features = dense_layers([text_features, image_features, condensed_likes])

age_group = tf.keras.layers.Dense(units=4, activation="softmax", name="age_group")(features)

model_age = tf.keras.Model(
    inputs=[text_features, image_features, likes_features],
    outputs= age_group
)    

model_age.compile(
    optimizer = tf.keras.optimizers.get({"class_name": "ADAM", #'ADAM'
                               "config": {"learning_rate": 0.0001}}),    
    loss = 'categorical_crossentropy',
    metrics = ['acc', 'categorical_accuracy']
)

print(model_age.summary())


Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
likes_features (InputLayer)     [(None, 100)]        0                                            
__________________________________________________________________________________________________
text_features (InputLayer)      [(None, 91)]         0                                            
__________________________________________________________________________________________________
image_features (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
likes_embedding_block (Sequenti (None, 800)          80000       likes_features[0][0]             
___________________________________________________________________________________________

In [106]:
#x_train_500, x_val_500, y_train_500, y_val_500

x_train_txt = x_train_100.iloc[:, :91].values
x_train_img = x_train_100.iloc[:, 91:156].values
x_train_lik = x_train_100.iloc[:, 156:].values

x_val_txt = x_val_100.iloc[:, :91].values
x_val_img = x_val_100.iloc[:, 91:156].values
x_val_lik = x_val_100.iloc[:, 156:].values

y_train_age = tf.keras.utils.to_categorical(y_train_100['age_group'].values)

y_val_age = tf.keras.utils.to_categorical(y_val_100['age_group'].values)


In [107]:
history_age_balanced_mild = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=16, epochs=30, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     class_weight=age_weights_dict_first3
                                    )

# validation baseline for age group:
print(y_val_100['age_group'].value_counts()[0]/y_val_100.shape[0])

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm



Train on 7600 samples, validate on 1900 samples
Epoch 1/30
7600/7600 [==============================] - 3s 420us/sample - loss: 6.6899 - acc: 0.4746 - categorical_accuracy: 0.4746 - val_loss: 4.6266 - val_acc: 0.5758 - val_categorical_accuracy: 0.5758
Epoch 2/30
7600/7600 [==============================] - 2s 245us/sample - loss: 3.7514 - acc: 0.5568 - categorical_accuracy: 0.5568 - val_loss: 3.2982 - val_acc: 0.6021 - val_categorical_accuracy: 0.6021
Epoch 3/30
7600/7600 [==============================] - 2s 245us/sample - loss: 3.1316 - acc: 0.5709 - categorical_accuracy: 0.5709 - val_loss: 3.0042 - val_acc: 0.5600 - val_categorical_accuracy: 0.5600
Epoch 4/30
7600/7600 [==============================] - 2s 240us/sample - loss: 2.8670 - acc: 0.5734 - categorical_accuracy: 0.5734 - val_loss: 2.7674 - val_acc: 0.5826 - val_categorical_accuracy: 0.5826
Epoch 5/30
7600/7600 [==============================] - 2s 241us/sample - loss: 2.6438 - acc: 0.5809 - categorical_accuracy: 0.5809 - va

array([[891, 234,   9,   0],
       [124, 289,  67,   0],
       [ 19,  87, 103,   0],
       [  7,  24,  46,   0]])

In [114]:
### Try different max number of liked pages per user!!

# model category age using embedding for likes

max_len = 200

image_features = tf.keras.Input([num_image_features], dtype=tf.float32, name="image_features")
text_features  = tf.keras.Input([num_text_features], dtype=tf.float32, name="text_features")
likes_features = tf.keras.Input([max_len], dtype=tf.int32, name="likes_features")

likes_embedding_block = tf.keras.Sequential(name="likes_embedding_block")
likes_embedding_block.add(tf.keras.layers.Embedding(10000, 8, input_length=max_len,
                                                    #embeddings_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001),
                                                   mask_zero=True))

### GlobalAveragePooling1D averages the embeddings of all pages liked by a user
#likes_embedding_block.add(tf.keras.layers.GlobalAveragePooling1D())

likes_embedding_block.add(tf.keras.layers.Flatten())

## Experimenting: Not helping at all... just learning the baseline
# Trying to reduce contribution of likes to balance out input modalities
#likes_embedding_block.add(tf.keras.layers.Dense(
#    units = 128, activation= 'tanh', 
#    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
## Experimenting

condensed_likes = likes_embedding_block(likes_features)

dense_layers = tf.keras.Sequential(name="dense_layers")
dense_layers.add(tf.keras.layers.Concatenate())
for i in range(num_layers):
    dense_layers.add(tf.keras.layers.Dense(
        units=dense_units,
        activation= 'tanh', #'tanh',
        kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg),      
        ))
        
    dense_layers.add(tf.keras.layers.Dropout(dropout_rate))
    
features = dense_layers([text_features, image_features, condensed_likes])

age_group = tf.keras.layers.Dense(units=4, activation="softmax", name="age_group")(features)

model_age = tf.keras.Model(
    inputs=[text_features, image_features, likes_features],
    outputs= age_group
)    

model_age.compile(
    optimizer = tf.keras.optimizers.get({"class_name": "ADAM", #'ADAM'
                               "config": {"learning_rate": 0.0001}}),    
    loss = 'categorical_crossentropy',
    metrics = ['acc', 'categorical_accuracy']
)

print(model_age.summary())



Model: "model_28"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
likes_features (InputLayer)     [(None, 200)]        0                                            
__________________________________________________________________________________________________
text_features (InputLayer)      [(None, 91)]         0                                            
__________________________________________________________________________________________________
image_features (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
likes_embedding_block (Sequenti (None, 1600)         80000       likes_features[0][0]             
___________________________________________________________________________________________

In [115]:
#x_train_500, x_val_500, y_train_500, y_val_500

x_train_txt = x_train_200.iloc[:, :91].values
x_train_img = x_train_200.iloc[:, 91:156].values
x_train_lik = x_train_200.iloc[:, 156:].values

x_val_txt = x_val_200.iloc[:, :91].values
x_val_img = x_val_200.iloc[:, 91:156].values
x_val_lik = x_val_200.iloc[:, 156:].values

y_train_age = tf.keras.utils.to_categorical(y_train_200['age_group'].values)

y_val_age = tf.keras.utils.to_categorical(y_val_200['age_group'].values)


In [116]:
history_age_balanced_mild = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=16, epochs=30, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     class_weight=age_weights_dict_first3
                                    )

# validation baseline for age group:
print(y_val_200['age_group'].value_counts()[0]/y_val_200.shape[0])

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm



Train on 7600 samples, validate on 1900 samples
Epoch 1/30
7600/7600 [==============================] - 3s 444us/sample - loss: 7.1958 - acc: 0.5289 - categorical_accuracy: 0.5289 - val_loss: 4.1471 - val_acc: 0.6105 - val_categorical_accuracy: 0.6105
Epoch 2/30
7600/7600 [==============================] - 2s 225us/sample - loss: 3.4122 - acc: 0.5662 - categorical_accuracy: 0.5662 - val_loss: 3.0775 - val_acc: 0.5647 - val_categorical_accuracy: 0.5647
Epoch 3/30
7600/7600 [==============================] - 2s 237us/sample - loss: 2.9789 - acc: 0.5632 - categorical_accuracy: 0.5632 - val_loss: 2.8278 - val_acc: 0.6105 - val_categorical_accuracy: 0.6105
Epoch 4/30
7600/7600 [==============================] - 2s 229us/sample - loss: 2.7497 - acc: 0.5726 - categorical_accuracy: 0.5726 - val_loss: 2.6132 - val_acc: 0.5932 - val_categorical_accuracy: 0.5932
Epoch 5/30
7600/7600 [==============================] - 2s 230us/sample - loss: 2.5541 - acc: 0.5853 - categorical_accuracy: 0.5853 - va

array([[898, 227,   9,   0],
       [147, 271,  62,   0],
       [ 16, 110,  83,   0],
       [  8,  28,  41,   0]])

In [118]:
### Try different max number of liked pages per user!!

# model category age using embedding for likes

max_len = 500

image_features = tf.keras.Input([num_image_features], dtype=tf.float32, name="image_features")
text_features  = tf.keras.Input([num_text_features], dtype=tf.float32, name="text_features")
likes_features = tf.keras.Input([max_len], dtype=tf.int32, name="likes_features")

likes_embedding_block = tf.keras.Sequential(name="likes_embedding_block")
likes_embedding_block.add(tf.keras.layers.Embedding(10000, 8, input_length=max_len,
                                                    #embeddings_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001),
                                                   mask_zero=True))

### GlobalAveragePooling1D averages the embeddings of all pages liked by a user
#likes_embedding_block.add(tf.keras.layers.GlobalAveragePooling1D())

likes_embedding_block.add(tf.keras.layers.Flatten())

## Experimenting: Not helping at all... just learning the baseline
# Trying to reduce contribution of likes to balance out input modalities
#likes_embedding_block.add(tf.keras.layers.Dense(
#    units = 128, activation= 'tanh', 
#    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
## Experimenting

condensed_likes = likes_embedding_block(likes_features)

dense_layers = tf.keras.Sequential(name="dense_layers")
dense_layers.add(tf.keras.layers.Concatenate())
for i in range(num_layers):
    dense_layers.add(tf.keras.layers.Dense(
        units=dense_units,
        activation= 'tanh', #'tanh',
        kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg),      
        ))
        
    dense_layers.add(tf.keras.layers.Dropout(dropout_rate))
    
features = dense_layers([text_features, image_features, condensed_likes])

age_group = tf.keras.layers.Dense(units=4, activation="softmax", name="age_group")(features)

model_age = tf.keras.Model(
    inputs=[text_features, image_features, likes_features],
    outputs= age_group
)    

model_age.compile(
    optimizer = tf.keras.optimizers.get({"class_name": "ADAM", #'ADAM'
                               "config": {"learning_rate": 0.0001}}),    
    loss = 'categorical_crossentropy',
    metrics = ['acc', 'categorical_accuracy']
)

print(model_age.summary())

#x_train_500, x_val_500, y_train_500, y_val_500

x_train_txt = x_train_500.iloc[:, :91].values
x_train_img = x_train_500.iloc[:, 91:156].values
x_train_lik = x_train_500.iloc[:, 156:].values

x_val_txt = x_val_500.iloc[:, :91].values
x_val_img = x_val_500.iloc[:, 91:156].values
x_val_lik = x_val_500.iloc[:, 156:].values

y_train_age = tf.keras.utils.to_categorical(y_train_500['age_group'].values)

y_val_age = tf.keras.utils.to_categorical(y_val_500['age_group'].values)

history_age_balanced_mild = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=64, epochs=50, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     class_weight=age_weights_dict_first3
                                    )

# validation baseline for age group:
print(y_val_500['age_group'].value_counts()[0]/y_val_500.shape[0])

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm


Model: "model_30"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
likes_features (InputLayer)     [(None, 500)]        0                                            
__________________________________________________________________________________________________
text_features (InputLayer)      [(None, 91)]         0                                            
__________________________________________________________________________________________________
image_features (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
likes_embedding_block (Sequenti (None, 4000)         80000       likes_features[0][0]             
___________________________________________________________________________________________

Epoch 32/50
7600/7600 [==============================] - 1s 107us/sample - loss: 1.4543 - acc: 0.6511 - categorical_accuracy: 0.6511 - val_loss: 1.4929 - val_acc: 0.6468 - val_categorical_accuracy: 0.6468
Epoch 33/50
7600/7600 [==============================] - 1s 108us/sample - loss: 1.4285 - acc: 0.6614 - categorical_accuracy: 0.6614 - val_loss: 1.4695 - val_acc: 0.6421 - val_categorical_accuracy: 0.6421
Epoch 34/50
7600/7600 [==============================] - 1s 103us/sample - loss: 1.3984 - acc: 0.6691 - categorical_accuracy: 0.6691 - val_loss: 1.4507 - val_acc: 0.6505 - val_categorical_accuracy: 0.6505
Epoch 35/50
7600/7600 [==============================] - 1s 107us/sample - loss: 1.3778 - acc: 0.6750 - categorical_accuracy: 0.6750 - val_loss: 1.4312 - val_acc: 0.6505 - val_categorical_accuracy: 0.6505
Epoch 36/50
7600/7600 [==============================] - 1s 108us/sample - loss: 1.3542 - acc: 0.6779 - categorical_accuracy: 0.6779 - val_loss: 1.4193 - val_acc: 0.6563 - val_cate

array([[883, 243,   8,   0],
       [159, 270,  51,   0],
       [ 14, 100,  95,   0],
       [  6,  43,  28,   0]])

In [120]:
### Try different max number of liked pages per user!!

# model category age using embedding for likes

max_len = 1000

image_features = tf.keras.Input([num_image_features], dtype=tf.float32, name="image_features")
text_features  = tf.keras.Input([num_text_features], dtype=tf.float32, name="text_features")
likes_features = tf.keras.Input([max_len], dtype=tf.int32, name="likes_features")

likes_embedding_block = tf.keras.Sequential(name="likes_embedding_block")
likes_embedding_block.add(tf.keras.layers.Embedding(10000, 8, input_length=max_len,
                                                    #embeddings_regularizer=tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001),
                                                   mask_zero=True))

### GlobalAveragePooling1D averages the embeddings of all pages liked by a user
#likes_embedding_block.add(tf.keras.layers.GlobalAveragePooling1D())

likes_embedding_block.add(tf.keras.layers.Flatten())

## Experimenting: Not helping at all... just learning the baseline
# Trying to reduce contribution of likes to balance out input modalities
#likes_embedding_block.add(tf.keras.layers.Dense(
#    units = 128, activation= 'tanh', 
#    kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg)))
## Experimenting

condensed_likes = likes_embedding_block(likes_features)

dense_layers = tf.keras.Sequential(name="dense_layers")
dense_layers.add(tf.keras.layers.Concatenate())
for i in range(num_layers):
    dense_layers.add(tf.keras.layers.Dense(
        units=dense_units,
        activation= 'tanh', #'tanh',
        kernel_regularizer=tf.keras.regularizers.L1L2(l1=l1_reg, l2=l2_reg),      
        ))
        
    dense_layers.add(tf.keras.layers.Dropout(dropout_rate))
    
features = dense_layers([text_features, image_features, condensed_likes])

age_group = tf.keras.layers.Dense(units=4, activation="softmax", name="age_group")(features)

model_age = tf.keras.Model(
    inputs=[text_features, image_features, likes_features],
    outputs= age_group
)    

model_age.compile(
    optimizer = tf.keras.optimizers.get({"class_name": "ADAM", #'ADAM'
                               "config": {"learning_rate": 0.0001}}),    
    loss = 'categorical_crossentropy',
    metrics = ['acc', 'categorical_accuracy']
)

print(model_age.summary())

#x_train_500, x_val_500, y_train_500, y_val_500

x_train_txt = x_train_1000.iloc[:, :91].values
x_train_img = x_train_1000.iloc[:, 91:156].values
x_train_lik = x_train_1000.iloc[:, 156:].values

x_val_txt = x_val_1000.iloc[:, :91].values
x_val_img = x_val_1000.iloc[:, 91:156].values
x_val_lik = x_val_1000.iloc[:, 156:].values

y_train_age = tf.keras.utils.to_categorical(y_train_1000['age_group'].values)

y_val_age = tf.keras.utils.to_categorical(y_val_1000['age_group'].values)

history_age_balanced_mild = model_age.fit([x_train_txt, x_train_img, x_train_lik], y_train_age, 
                                     shuffle=True, batch_size=64, epochs=55, verbose=1,
                                     validation_data=([x_val_txt, x_val_img, x_val_lik], y_val_age),
                                     class_weight=age_weights_dict_first3
                                    )

# validation baseline for age group:
print(y_val_1000['age_group'].value_counts()[0]/y_val_1000.shape[0])

## Create confusion matrix
# 0 = < 25
# 1 = 24-35
# 2 = 35-50
# 3 = 50+

y_pred = np.argmax(model_age.predict([x_val_txt, x_val_img, x_val_lik]), axis=1)
y_true = np.argmax(y_val_age, axis=1)

cm = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3])
cm



Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
likes_features (InputLayer)     [(None, 1000)]       0                                            
__________________________________________________________________________________________________
text_features (InputLayer)      [(None, 91)]         0                                            
__________________________________________________________________________________________________
image_features (InputLayer)     [(None, 65)]         0                                            
__________________________________________________________________________________________________
likes_embedding_block (Sequenti (None, 8000)         80000       likes_features[0][0]             
___________________________________________________________________________________________

Epoch 32/55
7600/7600 [==============================] - 1s 161us/sample - loss: 1.4558 - acc: 0.6420 - categorical_accuracy: 0.6420 - val_loss: 1.5006 - val_acc: 0.6289 - val_categorical_accuracy: 0.6289
Epoch 33/55
7600/7600 [==============================] - 1s 162us/sample - loss: 1.4325 - acc: 0.6475 - categorical_accuracy: 0.6475 - val_loss: 1.4824 - val_acc: 0.6289 - val_categorical_accuracy: 0.6289
Epoch 34/55
7600/7600 [==============================] - 1s 160us/sample - loss: 1.4099 - acc: 0.6537 - categorical_accuracy: 0.6537 - val_loss: 1.4651 - val_acc: 0.6326 - val_categorical_accuracy: 0.6326
Epoch 35/55
7600/7600 [==============================] - 1s 160us/sample - loss: 1.3921 - acc: 0.6625 - categorical_accuracy: 0.6625 - val_loss: 1.4504 - val_acc: 0.6316 - val_categorical_accuracy: 0.6316
Epoch 36/55
7600/7600 [==============================] - 1s 161us/sample - loss: 1.3738 - acc: 0.6692 - categorical_accuracy: 0.6692 - val_loss: 1.4360 - val_acc: 0.6274 - val_cate

array([[859, 266,   9,   0],
       [115, 302,  63,   0],
       [ 15, 104,  90,   0],
       [  7,  37,  33,   0]])